In [68]:
import pandas as pd
import datetime as dt
import seaborn as sns
import numpy as np

In [69]:
calories=pd.read_csv('./fitbit_preprocessed_data/calories.csv')
floors=pd.read_csv('./fitbit_preprocessed_data/floors.csv')
heart=pd.read_csv('./fitbit_preprocessed_data/heart.csv')
minutesFairlyActive=pd.read_csv('./fitbit_preprocessed_data/minutesFairlyActive.csv')
minutesLightlyActive=pd.read_csv('./fitbit_preprocessed_data/minutesLightlyActive.csv')
minutesSedentary=pd.read_csv('./fitbit_preprocessed_data/minutesSedentary.csv')
minutesVeryActive=pd.read_csv('./fitbit_preprocessed_data/minutesVeryActive.csv')
sleep=pd.read_csv('./fitbit_preprocessed_data/sleep.csv')
steps=pd.read_csv('./fitbit_preprocessed_data/steps.csv')

In [70]:
calories['date'] = pd.to_datetime(calories['datetime']).dt.date
calories['hour'] = pd.to_datetime(calories['datetime']).dt.hour

In [132]:
def date_hour(x):
    x['date'] = pd.to_datetime(x['datetime']).dt.date
    x['hour'] = pd.to_datetime(x['datetime']).dt.hour
    x = x[(x['hour']>8)]
    x['ampm']=0
    x.loc[x['hour']>11,'ampm'] = 1
    return x
def date_hour_sleep(x):
    x['date'] = pd.to_datetime(x['datetime']).dt.date
    x['hour'] = pd.to_datetime(x['datetime']).dt.hour
    x = x[(x['hour']<8)]
    x['ampm']=0
    x.loc[x['hour']>11,'ampm'] = 1
    return x

In [72]:
floors=date_hour(floors)
calories=date_hour(calories)
heart=date_hour(heart)
minutesFairlyActive=date_hour(minutesFairlyActive)
minutesLightlyActive=date_hour(minutesLightlyActive)
minutesSedentary=date_hour(minutesSedentary)
minutesVeryActive=date_hour(minutesVeryActive)
sleep=date_hour(sleep)


<ipython-input-71-76920b9ceb8f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['ampm']=0
/Users/yusuke-n/Library/Python/3.8/lib/python/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-71-76920b9ceb8f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

In [73]:
steps=date_hour(steps)

<ipython-input-71-76920b9ceb8f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['ampm']=0
/Users/yusuke-n/Library/Python/3.8/lib/python/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [74]:
floors_day_mean = floors.groupby(['user_id','date']).mean().reset_index()

In [75]:
def aggregate_feature(x):
    mean = x.groupby(['user_id','date']).mean().reset_index()
    median = x.groupby(['user_id','date']).median().reset_index()
    var = x.groupby(['user_id','date']).var().reset_index()
    merged_data = pd.merge(mean,median,on=['user_id','date'])
    merged_data = pd.merge(merged_data,var,on=['user_id','date'])
    merged_data['date_plus1'] = merged_data['date']+dt.timedelta(days=1)
    return merged_data
# def days_plus1(x):
#     for i in range(3):
#         x[i]['date_plus1'] = x[i]['date']+dt.timedelta(days=1)
#     return x

In [76]:
floors_day = aggregate_feature(floors)
calories_day = aggregate_feature(calories)
minutesFairlyActive_day = aggregate_feature(minutesFairlyActive)
minutesLightlyActive= aggregate_feature(minutesLightlyActive)
minutesSedentary= aggregate_feature(minutesSedentary)
minutesVeryActive= aggregate_feature(minutesVeryActive)
sleep= aggregate_feature(sleep)
steps= aggregate_feature(steps)

In [97]:
heart_day=aggregate_feature(heart)

In [94]:
floors_day = floors_day[['user_id', 'date', 'floor_x', 
        'floor_y', 'floor','date_plus1']] #floor_xがmean, floor_yがmedian, floorが
floors_day=floors_day.rename(columns={'floor_x':'floor_mean','floor_y':'floor_median','floor':'floor_var'})

In [93]:
calories_day = calories_day[['user_id', 'date', 'mets_x', 'calory_x',
        'mets_y','calory_y','mets', 'calory','date_plus1']]
calories_day=calories_day.rename(columns={'mets_x':'mets_mean','mets_y':'mets_median','mets':'mets_var',
                            'calory_x':'calory_mean','calory_y':'calory_median','calory':'calory_var'})

In [99]:
heart_day = heart_day[['user_id', 'date', 'heart_x', 
        'heart_y', 'heart','date_plus1']]
heart_day=heart_day.rename(columns={'heart_x':'heart_mean','heart_y':'heart_median','heart':'heart_var'})

In [107]:
minutesFairlyActive_day = aggregate_feature(minutesFairlyActive)
minutesFairlyActive_day = minutesFairlyActive_day[['user_id', 'date', 'minutesFairlyActive_x', 
        'minutesFairlyActive_y', 'minutesFairlyActive','date_plus1']]
minutesFairlyActive_day=minutesFairlyActive_day.rename(columns={'minutesFairlyActive_x':'minutesFairlyActive_mean','minutesFairlyActivet_y':'minutesFairlyActive_median','minutesFairlyActive':'minutesFairlyActive_var'})
minutesFairlyActive_day=minutesFairlyActive_day.rename(columns={'minutesFairlyActive_y':'minutesFairlyActive_median'})

In [119]:
minutesLightlyActive=pd.read_csv('./fitbit_preprocessed_data/minutesLightlyActive.csv')
minutesLightlyActive=date_hour(minutesLightlyActive)
minutesLightlyActive_day= aggregate_feature(minutesLightlyActive)
minutesLightlyActive_day = minutesLightlyActive_day[['user_id', 'date', 'minutesLightlyActive_x', 
        'minutesLightlyActive_y', 'minutesLightlyActive','date_plus1']]
minutesLightlyActive_day=minutesLightlyActive_day.rename(columns={'minutesLightlyActive_x':'minutesLightlyActive_mean','minutesLightlyActive_y':'minutesLightlyActive_median','minutesLightlyActive':'minutesLightlyActive_var'})

<ipython-input-71-76920b9ceb8f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['ampm']=0
/Users/yusuke-n/Library/Python/3.8/lib/python/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [125]:
minutesSedentary=pd.read_csv('./fitbit_preprocessed_data/minutesSedentary.csv')
minutesSedentary=date_hour(minutesSedentary)
minutesSedentary_day= aggregate_feature(minutesSedentary)
minutesSedentary_day = minutesSedentary_day[['user_id', 'date', 'minutesSedentary_x', 
        'minutesSedentary_y', 'minutesSedentary','date_plus1']]
minutesSedentary_day=minutesSedentary_day.rename(columns={'minutesSedentary_x':'minutesSedentary_mean','minutesSedentary_y':'minutesSedentary_median','minutesSedentary':'minutesSedentary_var'})

<ipython-input-71-76920b9ceb8f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['ampm']=0
/Users/yusuke-n/Library/Python/3.8/lib/python/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [127]:
minutesVeryActive=pd.read_csv('./fitbit_preprocessed_data/minutesVeryActive.csv')
minutesVeryActive=date_hour(minutesVeryActive)
minutesVeryActive_day= aggregate_feature(minutesVeryActive)
minutesVeryActive_day = minutesVeryActive_day[['user_id', 'date', 'minutesVeryActive_x', 
        'minutesVeryActive_y', 'minutesVeryActive','date_plus1']]
minutesVeryActive_day=minutesVeryActive_day.rename(columns={'minutesVeryActive_x':'minutesVeryActive_mean','minutesVeryActive_y':'minutesVeryActive_median','minutesVeryActive':'minutesVeryActive_var'})

<ipython-input-71-76920b9ceb8f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['ampm']=0
/Users/yusuke-n/Library/Python/3.8/lib/python/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [133]:
# sleep=date_hour(sleep)
sleep=pd.read_csv('./fitbit_preprocessed_data/sleep.csv')
sleep=date_hour_sleep(sleep)
sleep_day= aggregate_feature(sleep)
sleep_day = sleep_day[['user_id', 'date', 'sleep_x', 
        'sleep_y', 'sleep','date_plus1']]
sleep_day=sleep_day.rename(columns={'sleep_x':'sleep_mean','sleep_y':'sleep_median','sleep':'sleep_var'})

<ipython-input-132-bb78618472dd>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['ampm']=0
/Users/yusuke-n/Library/Python/3.8/lib/python/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [137]:
steps=pd.read_csv('./fitbit_preprocessed_data/steps.csv')
steps=date_hour(steps)
steps_day= aggregate_feature(steps)
steps_day = steps_day[['user_id', 'date', 'step_x', 
        'step_y', 'step','date_plus1']]
steps_day=steps_day.rename(columns={'step_x':'step_mean','step_y':'step_median','step':'step_var'})

<ipython-input-132-bb78618472dd>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['ampm']=0
/Users/yusuke-n/Library/Python/3.8/lib/python/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,user_id,date,step_mean,step_median,step_var,date_plus1
0,1,2021-01-07,0.000000,0.0,0.000000,2021-01-08
1,1,2021-01-08,0.000000,0.0,0.000000,2021-01-09
2,1,2021-01-09,0.000000,0.0,0.000000,2021-01-10
3,1,2021-01-10,0.000000,0.0,0.000000,2021-01-11
4,1,2021-01-11,0.000000,0.0,0.000000,2021-01-12
...,...,...,...,...,...,...
2641,100,2021-01-29,4.163333,0.0,188.628465,2021-01-30
2642,100,2021-01-30,9.767778,0.0,531.791397,2021-01-31
2643,100,2021-01-31,0.000000,0.0,0.000000,2021-02-01
2644,100,2021-02-01,0.000000,0.0,0.000000,2021-02-02
